FINAL PROJECT Module 2: Fit a logistic regression model or a linear regression model to the data source you identified in Module #1. Use the AIC and/or BIC criteria, and K-Fold cross validation to determine a best-fitting model. Interpret the coefficients of the model in the context of your chosen dataset and write a short paragraph describing your findings.

In [ ]:
import pandas as pd 

df = pd.read_csv("https://edg.epa.gov/EPADataCommons/public/OA/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import cross_val_score, train_test_split
from numpy import mean
from numpy import absolute
from numpy import sqrt
import statsmodels.api as sm

df

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,2.978361e+05
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,4.849451e+05
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,1.067059e+05
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,4.818284e+05
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,6.876848e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,7.803100e+11,7.803100e+11,78,30,961000,2,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,3414.446949,3.355857e+05
220736,220737,7.803100e+11,7.803100e+11,78,30,961000,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,2421.025608,2.924305e+05
220737,220738,7.803100e+11,7.803100e+11,78,30,961000,5,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,20,1,7.333333,1955.909418,1.619395e+05
220738,220739,7.803100e+11,7.803100e+11,78,30,960700,3,NaN,NaN,NaN,...,-99999.000000,-99999.000000,-99999.000000,1,1,10,1,4.000000,16896.768870,1.038966e+07


In [ ]:
from sklearn import preprocessing

#delete missing values (?)
df_clean=df.applymap(lambda x: np.nan if x == '?' else x).dropna()
print(df_clean.shape)

#drop city name
df_cleand=df_clean.drop(["CSA_Name",'CBSA_Name'],axis=1)

print(df_cleand)
# print(df.dtypes)

(167265, 117)
        OBJECTID       GEOID10       GEOID20  STATEFP  COUNTYFP  TRACTCE  \
0              1  4.811300e+11  4.811300e+11       48       113     7825   
1              2  4.811300e+11  4.811300e+11       48       113     7825   
2              3  4.811300e+11  4.811300e+11       48       113     7825   
3              4  4.811300e+11  4.811300e+11       48       113     7824   
4              5  4.811300e+11  4.811300e+11       48       113     7824   
...          ...           ...           ...      ...       ...      ...   
220646    220647  7.202100e+11  7.202100e+11       72        21    30200   
220647    220648  7.202100e+11  7.202100e+11       72        21    31401   
220648    220649  7.202100e+11  7.202100e+11       72        21    31202   
220649    220650  7.205320e+11  7.205320e+11       72        53   150400   
220650    220651  7.205320e+11  7.205320e+11       72        53   150104   

        BLKGRPCE    CSA     CBSA  CBSA_POP  ...         D5DRI          D5

In [ ]:
#select variables
data=df_cleand[['Ac_Land','CountHU','P_WrkAge','Pct_AO0','Pct_AO1','Pct_AO2p','Workers','R_MedWageWk']]
print(data)

           Ac_Land  CountHU  P_WrkAge   Pct_AO0   Pct_AO1  Pct_AO2p  Workers  \
0        73.595028    460.0     0.549  0.163121  0.092199  0.744681      412   
1       119.829909    409.0     0.466  0.000000  0.410758  0.589242      395   
2        26.367053    365.0     0.811  0.057751  0.434650  0.507599      463   
3       119.060687    384.0     0.638  0.000000  0.111979  0.888021      431   
4       169.927211    343.0     0.506  0.014577  0.195335  0.790087      579   
...            ...      ...       ...       ...       ...       ...      ...   
220646   34.442281    415.0     0.548  0.117470  0.475904  0.406627        0   
220647   27.042871    324.0     0.524  0.151203  0.309278  0.539519        0   
220648   49.273470    363.0     0.653  0.126280  0.447099  0.426621        0   
220649  111.259740    974.0     0.586  0.000000  0.744565  0.255435        0   
220650  153.582287    592.0     0.526  0.240000  0.594286  0.165714        0   

        R_MedWageWk  
0               1

In [ ]:
#separate x and y
y=df_cleand['NatWalkInd']
x=df_cleand.drop('NatWalkInd',axis=1)

#train test
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=1)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

cv = KFold(n_splits=10, random_state=1, shuffle=True)

#use k-fold CV to evaluate model
scores = cross_val_score(regr, data, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

#view mean absolute error
print("Score: %.4f:" % mean(scores))

Score: -3.1024:


In [ ]:
#AIC and BIC
import math

resid = y_test - y_pred
sse = sum(resid**2)
k=8
AIC=2*k-2*math.log(sse)
print(AIC)

BIC=k*math.log(len(X_test))-2*math.log(sse)
print(BIC)

-10.083419814518198
57.259754166549584


In [ ]:
regr.coef_

array([-1.01326131e-05, -3.20551286e-03,  6.28401665e+00,  6.05018650e+00,
        6.30133098e+00, -2.81774733e+00,  3.54874315e-03, -2.98511619e-03])